In [1]:
import pandas as pd
import numpy as np

In [2]:
df_original=pd.read_csv('../data/nyc_311_request_data_dec_2025.csv')
df=df_original.copy()

In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 323343 entries, 0 to 323342
Data columns (total 44 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   unique_key                      323343 non-null  int64  
 1   created_date                    323343 non-null  str    
 2   closed_date                     303798 non-null  str    
 3   agency                          323343 non-null  str    
 4   agency_name                     323343 non-null  str    
 5   complaint_type                  323343 non-null  str    
 6   descriptor                      319605 non-null  str    
 7   descriptor_2                    144037 non-null  str    
 8   location_type                   292803 non-null  str    
 9   incident_zip                    321092 non-null  float64
 10  incident_address                313995 non-null  str    
 11  street_name                     313989 non-null  str    
 12  cross_street_1             

In [4]:
df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,descriptor_2,location_type,incident_zip,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,67352282,2025-12-31T00:00:00.000,2026-01-09T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,NaN,Restaurant/Bar/Deli/Bakery,10003.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.726183,-73.986326,POINT (-73.986326176505 40.726183323174)
1,67334470,2025-12-30T23:59:35.000,2025-12-31T05:46:02.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,Residential Building/House,10472.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.832973,-73.876391,POINT (-73.876391113118 40.83297318207)
2,67343383,2025-12-30T23:59:31.000,2025-12-31T09:13:42.000,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,NO HEAT,RESIDENTIAL BUILDING,10467.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.884346,-73.878834,POINT (-73.878834348657 40.8843456267)
3,67337570,2025-12-30T23:59:28.000,2025-12-31T01:03:53.000,NYPD,New York City Police Department,Illegal Parking,Posted Parking Sign Violation,NaN,Street/Sidewalk,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.692591,-73.990963,POINT (-73.990963235578 40.692590626589)
4,67343958,2025-12-30T23:59:22.000,2025-12-31T15:30:10.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,NaN,Residential Building/House,11208.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.669321,-73.876734,POINT (-73.876734057579 40.669320633906)


In [5]:
null_percentage=(df.isnull().sum()*100/len(df)).sort_values(ascending=False).round(2)
null_percentage

taxi_company_borough              99.93
facility_type                     99.80
road_ramp                         99.79
bridge_highway_direction          99.75
due_date                          99.56
bridge_highway_segment            99.55
bridge_highway_name               99.55
taxi_pick_up_location             99.09
vehicle_type                      96.53
descriptor_2                      55.45
landmark                          45.09
intersection_street_1             39.90
intersection_street_2             39.86
cross_street_1                    37.67
cross_street_2                    37.66
location_type                      9.45
bbl                                7.85
closed_date                        6.04
city                               3.07
street_name                        2.89
incident_address                   2.89
resolution_description             2.09
resolution_action_updated_date     1.53
council_district                   1.35
descriptor                         1.16


In [6]:
# null_percentage.to_csv('../data/abcd.csv')

<small>

Always investigate columns that contain a high percentage of null values.  
In many cases, these nulls are **not errors** but are **logically dependent on other columns**, making them valid and meaningful.

**Example:**
- Columns: `vehicle_type`, `car_model`
- If `vehicle_type` includes categories such as *car*, *bike*, or *truck*,  
  then `car_model` will only be populated when the vehicle type is **car**.
- As a result, `car_model` may show a **high null percentage**, which is expected  
  because non-car records legitimately do not have a car model.

</small>


><small>
>
>💡 **Insight** 
>- For the columns with >95% nulls (e.g., taxi_company_borough, bridge_highway_name), the data is statistically >irrelevant. When a feature is missing for 99 out of 100 records, it ceases to be a variable and becomes an outlier >descriptor.
>
>- For the columns in the 30-55% range (e.g., intersection_street_1, landmark, cross_street_2), these are secondary >location descriptors. Since we have retained primary geolocation identifiers (likely incident_address, latitude, >longitude, or zip), these partial fields are redundant.
></small>

In [7]:
columns_with_null_perct_gt_30=list(null_percentage[null_percentage>30].index)

In [8]:
df.drop(columns=columns_with_null_perct_gt_30,inplace=True)

In [9]:
(df.isnull().sum()*100/len(df)).sort_values(ascending=False).round(2)

location_type                     9.45
bbl                               7.85
closed_date                       6.04
city                              3.07
street_name                       2.89
incident_address                  2.89
resolution_description            2.09
resolution_action_updated_date    1.53
council_district                  1.35
descriptor                        1.16
longitude                         0.96
latitude                          0.96
location                          0.96
x_coordinate_state_plane          0.96
y_coordinate_state_plane          0.95
incident_zip                      0.70
address_type                      0.40
park_facility_name                0.07
unique_key                        0.00
agency                            0.00
agency_name                       0.00
complaint_type                    0.00
created_date                      0.00
status                            0.00
community_board                   0.00
borough                  

### Analyze each column individually

> <small>
> 💡 <b>Insight</b><br>
> <i>These nulls act as status indicators. A null <b>closed_date</b> means the case is still open, and a missing resolution indicates the agency has not resolved it yet.</i><br><br>
> • closed_date — 6.04%<br>
> • resolution_description — 2.09%<br>
> • resolution_action_updated_date — 1.53%<br>
> <br>
> <b>Action:</b> Replace <code>resolution_description</code> nulls with “Unspecified” for graph analysis
> </small>


> <small>
> 💡 <b>Insight</b><br>
> <i>These define the “what” and “where” context. Losing these rows would waste valid data, so we label them as unknown.</i><br><br>
> • location_type — 9.45%<br>
> • descriptor — 1.16%<br>
> • address_type — 0.40%<br>
> <br>
> <b>Action:</b> Fill with “Unspecified”
> </small>


In [10]:
print(df.address_type.nunique())
df.address_type.value_counts()

5


address_type
ADDRESS         302427
INTERSECTION     16057
BLOCKFACE         2202
PLACE             1223
PLACENAME          146
Name: count, dtype: int64

In [11]:
print(df.location_type.nunique())
df.location_type.value_counts()

113


location_type
RESIDENTIAL BUILDING             102839
Street/Sidewalk                   74938
Residential Building/House        58937
Street                            20271
Sidewalk                          15770
                                  ...  
Single Room Occupancy (SRO)           1
Street Vendor                         1
Cafeteria - Private                   1
Correctional Facility - State         1
Theater                               1
Name: count, Length: 113, dtype: int64

> <small>
> 💡 <b>Insight</b><br>
> <i>These are core spatial identifiers used for mapping. Since the analysis relies on geospatial EDA, rows without coordinates or ZIP codes are statistically unusable.</i><br><br>
> • city — 3.07%<br>
> • incident_address — 2.89%<br>
> • council_district — 1.35%<br>
> • latitude / longitude — 0.96%<br>
> • location — 0.96%<br>
> • x_coordinate_state_plane / y_coordinate_state_plane — 0.96%<br>
> • incident_zip — 0.70%<br>
> <br>
> <b>Action:</b> Drop affected rows
> </small>


In [12]:
df[['street_name','incident_address','city','location','longitude','latitude','incident_zip','police_precinct',"community_board"]]

,street_name,incident_address,city,location,longitude,latitude,incident_zip,police_precinct,community_board
0,1 AVENUE,93 1 AVENUE,NEW YORK,POINT (-73.986326176505 40.726183323174),-73.986326,40.726183,10003.0,Precinct 9,03 MANHATTAN
1,STRATFORD AVENUE,1349 STRATFORD AVENUE,BRONX,POINT (-73.876391113118 40.83297318207),-73.876391,40.832973,10472.0,Precinct 43,09 BRONX
2,DEKALB AVENUE,3576 DEKALB AVENUE,BRONX,POINT (-73.878834348657 40.8843456267),-73.878834,40.884346,10467.0,Precinct 52,07 BRONX
3,COURT STREET,44 COURT STREET,BROOKLYN,POINT (-73.990963235578 40.692590626589),-73.990963,40.692591,11201.0,Precinct 84,02 BROOKLYN
4,MONTAUK AVENUE,373 MONTAUK AVENUE,BROOKLYN,POINT (-73.876734057579 40.669320633906),-73.876734,40.669321,11208.0,Precinct 75,05 BROOKLYN
...,...,...,...,...,...,...,...,...,...
323338,68 ROAD,144-43 68 ROAD,FLUSHING,POINT (-73.823795294901 40.732957874457),-73.823795,40.732958,11367.0,Precinct 107,08 QUEENS
323339,SHORE PARKWAY,2044 SHORE PARKWAY,BROOKLYN,POINT (-73.991218860912 40.586136875112),-73.991219,40.586137,11214.0,Precinct 60,13 BROOKLYN
323340,EAST 102 STREET,420 EAST 102 STREET,NEW YORK,POINT (-73.941138262803 40.78671817817),-73.941138,40.786718,10029.0,Precinct 23,11 MANHATTAN
323341,MCCLEAN AVENUE,200 MCCLEAN AVENUE,STATEN ISLAND,POINT (-74.06883485383 40.597249502152),-74.068835,40.597250,10305.0,Precinct 122,02 STATEN ISLAND


> <small>
> 💡 <b>Insight</b><br>
> <i>The column <code>location</code> duplicates information already available in <code>longitude</code> and <code>latitude</code>, making it redundant.</i><br>
> <b>Action:</b> Drop this column
> </small>


><small>
>
>💡 **Insight** 
>
>**incident_address** is more granular and contains the complete information than **street_name**. As shown in above data, incident_address includes the specific building number plus the street name (e.g., "93 1 AVENUE"), whereas street_name is just the >generic road (e.g., "1 AVENUE").
>
></small>

In [13]:
print(df.park_facility_name.nunique())

df.park_facility_name.value_counts()

222


park_facility_name
Unspecified                                           322572
Marcus Garvey Park                                       110
St. Nicholas Park                                         25
St. Mary's Park (Bronx)                                   24
Maria Hernandez Park                                      15
                                                       ...  
Harmony Park                                               1
Willoughby Playground                                      1
Bushwick Leaders High School for Academic Excellen         1
Constance Baker Motley Recreation Center                   1
Cypress Hills Ascend Charter School                        1
Name: count, Length: 222, dtype: int64

> <small>
> 💡 <b>Insight</b><br>
> <i>The column <code>park_facility_name</code> contains mostly <code>Unspecified</code> values (over 322k), with only a few rows containing actual park names. This makes the column largely uninformative for analysis.</i><br>
> <b>Action:</b> Drop this column
> </small>


In [14]:
df[['park_borough','borough']]

,park_borough,borough
0,MANHATTAN,MANHATTAN
1,BRONX,BRONX
2,BRONX,BRONX
3,BROOKLYN,BROOKLYN
4,BROOKLYN,BROOKLYN
...,...,...
323338,QUEENS,QUEENS
323339,BROOKLYN,BROOKLYN
323340,MANHATTAN,MANHATTAN
323341,STATEN ISLAND,STATEN ISLAND


> <small>
> 💡 <b>Insight</b><br>
> <i>The column <code>park_borough</code> is redundant and duplicates the information in <code>borough</code>.</i><br>
> <b>Action:</b> Drop this column
> </small>


In [15]:
df[['agency','agency_name']]

,agency,agency_name
0,DOHMH,Department of Health and Mental Hygiene
1,NYPD,New York City Police Department
2,HPD,Department of Housing Preservation and Develop...
3,NYPD,New York City Police Department
4,NYPD,New York City Police Department
...,...,...
323338,NYPD,New York City Police Department
323339,NYPD,New York City Police Department
323340,NYPD,New York City Police Department
323341,NYPD,New York City Police Department


> <small>
> 💡 <b>Insight</b><br>
> <i>The column <code>agency_name</code> is redundant, as it duplicates <code>agency</code>. The acronym is sufficient.</i><br>
> <b>Action:</b> Drop this column
> </small>


In [16]:
print(df.complaint_type.nunique())
df.complaint_type.value_counts()

170


complaint_type
HEAT/HOT WATER             61940
Noise - Residential        56344
Illegal Parking            42775
Blocked Driveway           15898
UNSANITARY CONDITION        9472
                           ...  
Radioactive Material           1
Wayfinding                     1
Taxi Licensee Complaint        1
Borough Office                 1
Tanning                        1
Name: count, Length: 170, dtype: int64

In [17]:
print(df.police_precinct.nunique())
df.police_precinct.value_counts()

78


police_precinct
Precinct 47     36216
Precinct 44      8090
Precinct 52      8017
Precinct 75      7558
Precinct 70      7131
                ...  
Precinct 69      1639
Precinct 17      1621
Precinct 26      1313
Precinct 100      984
Precinct 22       130
Name: count, Length: 78, dtype: int64

In [18]:
print(df.bbl.nunique())
df.bbl.unique()

85573


array([1.00447003e+09, 2.03866005e+09, 2.03329002e+09, ...,
       3.06133000e+09, 4.08892012e+09, 4.06533005e+09], shape=(85574,))

<small>
Borough-Block-Lot - BBL is the unique identifier for NYC tax lots. We can keep this if we want to join this data with external housing data. But for this Analysis, we don't need this
</small>

In [19]:
print(df.location_type.nunique())
df.location_type.value_counts()

113


location_type
RESIDENTIAL BUILDING             102839
Street/Sidewalk                   74938
Residential Building/House        58937
Street                            20271
Sidewalk                          15770
                                  ...  
Single Room Occupancy (SRO)           1
Street Vendor                         1
Cafeteria - Private                   1
Correctional Facility - State         1
Theater                               1
Name: count, Length: 113, dtype: int64

In [20]:
print(df.community_board.nunique())
df.community_board.value_counts()

77


community_board
12 BRONX            36207
12 MANHATTAN         9911
04 BRONX             8084
07 BRONX             8011
05 BROOKLYN          7463
                    ...  
26 BRONX               17
95 STATEN ISLAND       13
56 BROOKLYN             4
84 QUEENS               2
27 BRONX                1
Name: count, Length: 77, dtype: int64

In [21]:
df.incident_zip.nunique()
df.incident_zip.value_counts()

incident_zip
10466.0    32556
11226.0     5738
10467.0     4805
10458.0     4730
10456.0     4710
           ...  
17035.0        1
10152.0        1
89145.0        1
10045.0        1
10107.0        1
Name: count, Length: 235, dtype: int64

In [22]:
df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'address_type', 'city', 'status',
       'resolution_description', 'resolution_action_updated_date',
       'community_board', 'council_district', 'police_precinct', 'bbl',
       'borough', 'x_coordinate_state_plane', 'y_coordinate_state_plane',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'latitude', 'longitude', 'location'],
      dtype='str')

In [23]:
df[['location','latitude','longitude']].head()

,location,latitude,longitude
0,POINT (-73.986326176505 40.726183323174),40.726183,-73.986326
1,POINT (-73.876391113118 40.83297318207),40.832973,-73.876391
2,POINT (-73.878834348657 40.8843456267),40.884346,-73.878834
3,POINT (-73.990963235578 40.692590626589),40.692591,-73.990963
4,POINT (-73.876734057579 40.669320633906),40.669321,-73.876734


In [24]:
df['open_data_channel_type'].value_counts()

open_data_channel_type
ONLINE     148165
MOBILE      86834
PHONE       68195
UNKNOWN     20149
Name: count, dtype: int64

##### Invalid data check

TypeError: operation 'sub' not supported for dtype 'str' with dtype 'str'